In [1]:
%pip install numpy matplotlib deepxde

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.4/184.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 9.1 MB/s eta 0:00:00


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import deepxde as dde

No backend selected.
Finding available backend...


Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.
Instructions for updating:
non-resource variables are not supported in the long term


Found tensorflow.compat.v1
Setting the default backend to "tensorflow.compat.v1". You can change it in the ~/.deepxde/config.json file or export the DDE_BACKEND environment variable. Valid options are: tensorflow.compat.v1, tensorflow, pytorch, jax, paddle (all lowercase)


Enable just-in-time compilation with XLA.



In [6]:
#dde.backend.set_default_backend('pytorch')
#%cat ~/.deepxde/config.json

In [7]:
#PARÂMETROS DOMINIO
L = 20
TF = 80

#PARAMETROS
D = 0.0139
P = 0.0453

In [ ]:
Dl = D / (P * L**2)

#DECLARAÇÃO DOMÍNIO ESPACIAL E TEMPORAL
geom = dde.geometry.Interval(0, 1)
time = dde.geometry.TimeDomain(0, TF * P)
geomtime = dde.geometry.GeometryXTime(geom, time)

#EQUAÇÃO RESIDUAL
def pde(x, c):

    c_t = dde.grad.jacobian(c, x, i=0, j=1)
    c_xx = dde.grad.hessian(c, x, i=0, j=0)
    return c_t - (Dl * c_xx + c)

#CONDIÇÃO INICIAL
def ic_func(x): return np.exp(-100 * (x[:, 0:1] - 0.5)**2)
IC = dde.icbc.IC(geomtime, ic_func, lambda _, on_initial: on_initial)

#CONDIÇÃO DE CONTORNO
def bc_func(x): return 0
BC = dde.icbc.NeumannBC(geomtime, bc_func, lambda _, on_boundary: on_boundary)

#DECLARAÇÃO TREINAMENTO
DATA = dde.data.TimePDE(
    geomtime,
    pde,
    [IC, BC],
    num_domain = 2_000,
    num_boundary = 500,
    num_initial = 1_000,
    num_test = 1_000
)

NET = dde.nn.FNN(
    [2] + [150] * 3 + [1],
    'tanh',
    "Glorot uniform"
)

MODEL = dde.Model(DATA, NET)
MODEL.compile("adam", lr=1e-3)
losshistory, train_state = MODEL.train(
    50_000,
    #callbacks=[checker]
)

Compiling model...
Building feed-forward neural network...
'build' took 0.054439 s

'compile' took 0.714612 s

Training model...

Step      Train loss                        Test loss                         Test metric
0         [3.18e-01, 1.65e-01, 1.34e-02]    [3.87e-01, 1.65e-01, 1.34e-02]    []  
1000      [2.09e-02, 1.64e-02, 1.31e-04]    [3.71e-02, 1.64e-02, 1.31e-04]    []  
2000      [2.15e-02, 6.93e-03, 4.36e-04]    [3.98e-02, 6.93e-03, 4.36e-04]    []  
3000      [2.12e-02, 6.71e-03, 4.29e-04]    [3.90e-02, 6.71e-03, 4.29e-04]    []  
4000      [2.05e-02, 6.08e-03, 2.26e-05]    [3.80e-02, 6.08e-03, 2.26e-05]    []  
5000      [1.78e-02, 6.60e-03, 2.80e-04]    [3.27e-02, 6.60e-03, 2.80e-04]    []  
6000      [1.52e-02, 4.28e-03, 3.08e-05]    [2.77e-02, 4.28e-03, 3.08e-05]    []  
7000      [1.19e-02, 3.75e-03, 2.03e-05]    [2.16e-02, 3.75e-03, 2.03e-05]    []  
8000      [1.34e-02, 3.33e-03, 5.85e-04]    [2.46e-02, 3.33e-03, 5.85e-04]    []  
9000      [5.85e-03, 1.72e-03, 5.

In [ ]:
import torch

X = torch.linspace(0, 1, 1000)
T = torch.linspace(0, TF * P, 80)

INPUT = torch.cartesian_prod(X, T)
OUT = MODEL.predict(INPUT)
OUT = OUT.reshape(1000, 80).T

fig, ax = plt.subplots()

plt.pcolormesh(X * L, T / P, OUT, cmap='jet')
plt.title('Murray PINNs')
plt.xlabel('x (mm)')
plt.ylabel('t (dias)')
plt.colorbar()
plt.show()

plt.savefig('figuras/murray_pinns_3x150x2000.png')